In [1]:
### Defining reverse complement function

def reverse_complement(DNA):
    reverse = DNA.upper()[::-1]
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    return(''.join([complement[base] for base in reverse]))

### Defining function that returns 8 base pair sliding window from input of 15 bp sequence, major allele, minor allele

def eightbp_major_minor(seq_15, major, minor):
    major_15 = seq_15[0:7] + major + seq_15[8:]
    minor_15 = seq_15[0:7] + minor + seq_15[8:]
    list_of_major_minor_pairs = []
    for i in range(0, 8):
        list_of_major_minor_pairs.append([major_15[i:(i+8)].upper(), minor_15[i:(i+8)].upper()])
    return(list_of_major_minor_pairs)

In [2]:
### Importing RBC SNPs

import re

test = open("RBC_SNPs_single_30072018.txt", "r")
count = 0
rsIDs = {}
# list_RBC_rsIDs = []
# key_rsIDs = str()
checkRepeats = []

for readLine in test:
    if count == 0:
        count += 1
        continue
    if count > 0:
        count += 1
        key_rsIDs = (readLine.split())[1]
        try:
            len((readLine.split())[4]) == 41 & len((readLine.split())[5]) == 41 
        except:
            print(key_rsIDs + "is not of length 41")
        if key_rsIDs in rsIDs:
            checkRepeats.append(key_rsIDs)
        rsIDs[key_rsIDs] = [(readLine.split())[1], ((readLine.split())[4])[13:28].upper(), ((readLine.split())[5])[13:28].upper()]

In [3]:
### Check for Signalling Center SNPs

file_Signalling_SNPs = open("Signalling_Ctr_SNPs_04092018.txt", "r")
list_Signalling_SNPs = []
hash_Signalling_SNPs = {}

for readLine in file_Signalling_SNPs:
    list_Signalling_SNPs.append(readLine.rstrip())

for i in list_Signalling_SNPs:
    if i in rsIDs:
        hash_Signalling_SNPs[i] = rsIDs[i]

### Create sliding window of 734 Signalling Centre SNPs

hash_SigCtr_SNPs_sliding_window = {}

for k, v in hash_Signalling_SNPs.items():
    hash_SigCtr_SNPs_sliding_window[k] = []
    for i in range(0, 8):
        hash_SigCtr_SNPs_sliding_window[k].append([hash_Signalling_SNPs[k][1][i:(i+8)].upper(), hash_Signalling_SNPs[k][2][i:(i+8)].upper()])
print(len(hash_SigCtr_SNPs_sliding_window))

734


In [4]:
### Getting all missing Signalling Ctr SNPs from dbSNP, with 5 indels removed - recovered 65 out of 70 absent SNPs
### Of these remaining 65 SNPs - picked out the minor allele that had some allele frequency information

import re

file_vcf_absent_SNPs = open("absent_SigCtr_RBC_SNPs_caf.recode.vcf", "r")
count = 0
hash_vcf_absent_SNPs = {}

for readLine in file_vcf_absent_SNPs:
    count += 1
    if (count >= 58):
        list_of_SNP_info = readLine.split()
        if len(list_of_SNP_info[3]) > 1:
            print("Line " + str(count) + " removed")
            continue
        if len(list_of_SNP_info[4]) == 3:
            match_test = re.search("CAF=(.*),(.*),(.*)$", list_of_SNP_info[7])
            if match_test.group(2) != ".":
                minor_allele = list_of_SNP_info[4][0]
                caf = "CAF=" + str(match_test.group(1)) + "," + str(match_test.group(2))
            if match_test.group(3) != ".":
                minor_allele = list_of_SNP_info[4][2]
                caf = "CAF=" + str(match_test.group(1)) + "," + str(match_test.group(3))
            hash_vcf_absent_SNPs[list_of_SNP_info[2]] = (list_of_SNP_info[0], list_of_SNP_info[1],
                                                           list_of_SNP_info[2], list_of_SNP_info[3],
                                                           minor_allele, caf)
            continue
        if len(list_of_SNP_info[4]) == 5:
            match_test = re.search("CAF=(.*),(.*),(.*),(.*)$", list_of_SNP_info[7])
            if match_test.group(2) != ".":
                minor_allele = list_of_SNP_info[4][0]
                caf = "CAF=" + str(match_test.group(1)) + "," + str(match_test.group(2))
            if match_test.group(3) != ".":
                minor_allele = list_of_SNP_info[4][2]
                caf = "CAF=" + str(match_test.group(1)) + "," + str(match_test.group(3))
            if match_test.group(4) != ".":
                minor_allele = list_of_SNP_info[4][4]
                caf = "CAF=" + str(match_test.group(1)) + "," + str(match_test.group(4))
            hash_vcf_absent_SNPs[list_of_SNP_info[2]] = (list_of_SNP_info[0], list_of_SNP_info[1],
                                                           list_of_SNP_info[2], list_of_SNP_info[3],
                                                           minor_allele, caf)
            continue
        else:
            hash_vcf_absent_SNPs[list_of_SNP_info[2]] = (list_of_SNP_info[0], list_of_SNP_info[1],
                                                           list_of_SNP_info[2], list_of_SNP_info[3],
                                                           list_of_SNP_info[4], list_of_SNP_info[7])

print(len(hash_vcf_absent_SNPs))

Line 67 removed
Line 85 removed
Line 95 removed
Line 120 removed
Line 126 removed
65


In [5]:
### Using Biopython SeqIO module to obtain 15-bp window with each SNP at the centre - 
### for obtaining sliding window of 8-mers for SNPs

from Bio import SeqIO

list_chr = []

for i in range(1, 23):
    list_chr.append(i)
list_chr.append("X")
list_chr.append("Y")

hash_RBC_SigCtr_SNPs_8mer_major_minor = {}

for str_chr in list_chr:
    str_file_name_chr = "chr" + str(str_chr) + ".fa"
    for seq_record in SeqIO.parse(str_file_name_chr, "fasta"):
        for k, v in hash_vcf_absent_SNPs.items():
            if v[0] == str(str_chr):
                chrpos = int(v[1])
                seq_15_current = str(seq_record.seq[(chrpos-8):(chrpos+7)]).upper()
                if len(v[4]) == 1:
                    hash_RBC_SigCtr_SNPs_8mer_major_minor[k] = eightbp_major_minor(seq_15_current, v[3], v[4])
                else:
                    print(hash_vcf_absent_SNPs[k])

In [6]:
### Combining the existing list of Signalling Ctr RBC SNPs, with those that were absent in the RBC SNPs list - total 799

hash_combined_SigCtr_SNPs = {}

for k, v in hash_RBC_SigCtr_SNPs_8mer_major_minor.items():
    hash_combined_SigCtr_SNPs[k] = hash_RBC_SigCtr_SNPs_8mer_major_minor[k]

for k, v in hash_SigCtr_SNPs_sliding_window.items():
    hash_combined_SigCtr_SNPs[k] = hash_SigCtr_SNPs_sliding_window[k]

print(len(hash_combined_SigCtr_SNPs))

799


In [7]:
### Extracting relevant information regarding rsIDs of interest from Framingham Heart Study cis-eQTL Excel spreadsheet

import pandas as pd, os, re

hash_rsIDs_with_ciseQTL_gene = {}
hash_rsIDs_with_ciseQTL_exon = {}

pd_df_Framingham = pd.read_excel("FHS_SNP_SignalingCenters_new.xlsx", sheetname = 1, header = None, skiprows = 4)
list_rsID = pd_df_Framingham[0].tolist()
list_SNP_Fx_Allele = pd_df_Framingham[6].tolist()
list_SNP_Non_Fx_Allele = pd_df_Framingham[7].tolist()
list_T = pd_df_Framingham[22].tolist()
list_log10FDR = pd_df_Framingham[24].tolist()

for i in range(0, 873):
    hash_rsIDs_with_ciseQTL_gene[list_rsID[i]] = (list_SNP_Fx_Allele[i], list_SNP_Non_Fx_Allele[i], list_T[i], list_log10FDR[i])

exon_pd_df_Framingham = pd.read_excel("FHS_SNP_SignalingCenters_new.xlsx", sheetname = 3, header = None, skiprows = 4)
exon_list_rsID = exon_pd_df_Framingham[0].tolist()
exon_list_SNP_Fx_Allele = exon_pd_df_Framingham[6].tolist()
exon_list_SNP_Non_Fx_Allele = exon_pd_df_Framingham[7].tolist()
exon_list_T = exon_pd_df_Framingham[22].tolist()
exon_list_log10FDR = exon_pd_df_Framingham[24].tolist()

for i in range(0, 873):
    hash_rsIDs_with_ciseQTL_exon[exon_list_rsID[i]] = (exon_list_SNP_Fx_Allele[i], exon_list_SNP_Non_Fx_Allele[i], exon_list_T[i], exon_list_log10FDR[i])

In [8]:
import os

### Sliding window of 3,263 RBC SNPs

hash_RBC_SNPs_sliding_window = {}

for k, v in rsIDs.items():
    hash_RBC_SNPs_sliding_window[k] = []
    for i in range(0, 8):
        hash_RBC_SNPs_sliding_window[k].append([rsIDs[k][1][i:(i+8)].upper(), rsIDs[k][2][i:(i+8)].upper()])

### Generating dictionary versions of individual PBM datasets of interest for evaluating perturbation of TF binding
### The reverse_complement() function is needed to ensure that all forward and reverse 8-mer orientations are present.

hash_all_PBMs_with_Escores = {}

for file in os.listdir("Individual_PBMs_Escores/"):
    if re.search("_Escore.txt", file):
        m = re.match("(.*)_Escore\.txt", file)
        str_PBM_name = (m.group(1))
        PBM_file = open("Individual_PBMs_Escores/" + file, "r")
        hash_all_PBMs_with_Escores.setdefault(str_PBM_name, {})
        for readLine in PBM_file:
            key = readLine.rstrip("\n").split("\t")[0]
            value = readLine.rstrip("\n").split("\t")[1]
            key_RC = reverse_complement(key)
            hash_all_PBMs_with_Escores[str_PBM_name][key] = value
            hash_all_PBMs_with_Escores[str_PBM_name][key_RC] = value
        PBM_file.close()

In [10]:
### Analysing 3,263 RBC SNPs for SNPs that perturb TF binding (based on individual PBM datasets)

hash_PBM_statistics = {}
list_rsIDs_PBMs_of_interest = []

for str_PBM_name in hash_all_PBMs_with_Escores.keys():

    count_loss_binding = 0
    count_surprise_no_loss_binding = 0
    count_gain_binding = 0
    count_surprise_no_gain_binding = 0
    testcount = 0
    
    for k, v in hash_RBC_SNPs_sliding_window.items():        
        testcount8mer = 0
        for i in range(0, 8):
            allele1_key = v[i][0]
            allele2_key = v[i][1]
            if allele1_key not in hash_all_PBMs_with_Escores[str_PBM_name]:
                print(allele1_key)
                continue
            if allele2_key not in hash_all_PBMs_with_Escores[str_PBM_name]:
                print(allele2_key)
                continue
            if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) > 0.35) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) < 0.3):
                count_loss_binding += 1
                testcount8mer += 1
#                print(k + "\t" + str_PBM_name + "\t" + allele1_key + "\t" + allele2_key + "\t" + str(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) + "\t" + str(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]))
                list_rsIDs_PBMs_of_interest.append([k, str_PBM_name])
            if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) > 0.3) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) <= float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key])):
                count_surprise_no_loss_binding += 1
            if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) > 0.35) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) < 0.3):
                count_gain_binding += 1
                testcount8mer += 1
#                print(k + "\t" + str_PBM_name + "\t" + allele1_key + "\t" + allele2_key + "\t" + str(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) + "\t" + str(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]))
                list_rsIDs_PBMs_of_interest.append([k, str_PBM_name])
            if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) > 0.3) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) <= float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key])):
                count_surprise_no_gain_binding += 1
        if testcount8mer > 0:
            testcount += 1

    hash_PBM_statistics[str_PBM_name] = (str_PBM_name, len(hash_RBC_SNPs_sliding_window)*8, count_loss_binding, count_gain_binding, testcount)
#    print(hash_PBM_statistics[str_PBM_name])
print(hash_PBM_statistics)

{'Egr1_2580_contig8mers': ('Egr1_2580_contig8mers', 26104, 466, 466, 439), 'EGR2_REF_R1_8mers': ('EGR2_REF_R1_8mers', 26104, 500, 504, 442), 'Gata3_1024_contig8mers': ('Gata3_1024_contig8mers', 26104, 425, 344, 326), 'Gata4_8mers_11111111': ('Gata4_8mers_11111111', 26104, 201, 145, 172), 'Gata5_3768_contig8mers': ('Gata5_3768_contig8mers', 26104, 291, 202, 209), 'Gata6_3769_contig8mers': ('Gata6_3769_contig8mers', 26104, 365, 267, 235), 'GATA_average': ('GATA_average', 26104, 276, 181, 184), 'Gli1_v015681_8mers_11111111': ('Gli1_v015681_8mers_11111111', 26104, 455, 444, 487), 'Gli2_v015681_8mers_11111111': ('Gli2_v015681_8mers_11111111', 26104, 472, 450, 504), 'Gli3_v015681_8mers_11111111': ('Gli3_v015681_8mers_11111111', 26104, 506, 499, 517), 'HES1_R1': ('HES1_R1', 26104, 190, 148, 196), 'HES5_R1': ('HES5_R1', 26104, 364, 321, 392), 'HES7_R1': ('HES7_R1', 26104, 318, 287, 328), 'KLF1_REF_R2_8mers': ('KLF1_REF_R2_8mers', 26104, 224, 182, 226), 'Rara_1051_contig8mers': ('Rara_1051_cont

In [11]:
### Using individual PBMs to scan 3263 RBC SNPs - what types of binding changes do we see? 
### Criteria: E-score > 0.35 for one allele and E-score < 0.3 for other allele

hash_PBM_statistics = {}

for str_PBM_name in hash_all_PBMs_with_Escores.keys():

    count_loss_binding = 0
    count_surprise_no_loss_binding = 0
    count_gain_binding = 0
    count_surprise_no_gain_binding = 0
    list_eightmer_change = []
    
    for k, v in hash_RBC_SNPs_sliding_window.items():
        eight8mer_change = 0
        for i in range(0, 8):
            allele1_key = v[i][0]
            allele2_key = v[i][1]
            if allele1_key not in hash_all_PBMs_with_Escores[str_PBM_name]:
                print(allele1_key)
                continue
            if allele2_key not in hash_all_PBMs_with_Escores[str_PBM_name]:
                print(allele2_key)
                continue
            if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) > 0.35) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) < 0.3):
                count_loss_binding += 1
                eight8mer_change += 1
            if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) > 0.3) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) <= float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key])):
                count_surprise_no_loss_binding += 1
            if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) > 0.35) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) < 0.3):
                count_gain_binding += 1
                eight8mer_change += 1
            if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) > 0.3) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) <= float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key])):
                count_surprise_no_gain_binding += 1
        list_eightmer_change.append(eight8mer_change)

#    testing = open(str_PBM_name + ".txt", "w")
#    for item in list_eightmer_change:
#        testing.write("%s\n" % item)
    hash_PBM_statistics[str_PBM_name] = (str_PBM_name, len(hash_RBC_SNPs_sliding_window)*8, count_loss_binding, count_gain_binding, list_eightmer_change, sum(list_eightmer_change))
    count_item_eightmer = 0
    for item_eightmer in list_eightmer_change:
        if item_eightmer > 0:
            count_item_eightmer += 1
    print(str_PBM_name, len(hash_RBC_SNPs_sliding_window)*8, count_loss_binding, count_gain_binding, count_item_eightmer, sum(list_eightmer_change))
#    testing.close()

Egr1_2580_contig8mers 26104 466 466 439 932
EGR2_REF_R1_8mers 26104 500 504 442 1004
Gata3_1024_contig8mers 26104 425 344 326 769
Gata4_8mers_11111111 26104 201 145 172 346
Gata5_3768_contig8mers 26104 291 202 209 493
Gata6_3769_contig8mers 26104 365 267 235 632
GATA_average 26104 276 181 184 457
Gli1_v015681_8mers_11111111 26104 455 444 487 899
Gli2_v015681_8mers_11111111 26104 472 450 504 922
Gli3_v015681_8mers_11111111 26104 506 499 517 1005
HES1_R1 26104 190 148 196 338
HES5_R1 26104 364 321 392 685
HES7_R1 26104 318 287 328 605
KLF1_REF_R2_8mers 26104 224 182 226 406
Rara_1051_contig8mers 26104 231 265 271 496
Rarb_8mers_11111111 26104 133 162 138 295
Rarg_8mers_11111111 26104 158 206 177 364
Rxra_1035_contig8mers 26104 259 288 284 547
Rxrg_8mers_11111111 26104 141 183 180 324
Sfpi1_1034_contig8mers 26104 380 460 360 840
Smad3_3805_contig8mers 26104 332 354 306 686
TCF4_R1 26104 142 145 140 287


In [12]:
### Getting average GATA PBM dataset to represent GATA3, GATA4, GATA5 and GATA6 datasets

hash_all_PBMs_with_Escores["GATA_average"] = {}
#file_GATA_average = open("GATA_average_Escore.txt", "w")

for i in hash_all_PBMs_with_Escores["Gata3_1024_contig8mers"]:
    Escore_Gata3 = float(hash_all_PBMs_with_Escores["Gata3_1024_contig8mers"][i])
    Escore_Gata4 = float(hash_all_PBMs_with_Escores["Gata4_8mers_11111111"][i])
    Escore_Gata5 = float(hash_all_PBMs_with_Escores["Gata5_3768_contig8mers"][i])
    Escore_Gata6 = float(hash_all_PBMs_with_Escores["Gata6_3769_contig8mers"][i])
    average_Escore_Gata = (Escore_Gata3 + Escore_Gata4 + Escore_Gata5 + Escore_Gata6)/4
    hash_all_PBMs_with_Escores["GATA_average"][i] = average_Escore_Gata
#    file_GATA_average.write(i + "\t" + str(average_Escore_Gata) + "\n")
#file_GATA_average.close()

### GATA average for 3263 RBC SNPs

str_PBM_name = "GATA_average"

count_loss_binding = 0
count_surprise_no_loss_binding = 0
count_gain_binding = 0
count_surprise_no_gain_binding = 0

for k, v in hash_RBC_SNPs_sliding_window.items():
    for i in range(0, 8):
        allele1_key = v[i][0]
        allele2_key = v[i][1]
        if allele1_key not in hash_all_PBMs_with_Escores[str_PBM_name]:
            print(allele1_key)
            continue
        if allele2_key not in hash_all_PBMs_with_Escores[str_PBM_name]:
            print(allele2_key)
            continue
        if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) > 0.35) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) < 0.3):
            count_loss_binding += 1
        if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) > 0.3) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) <= float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key])):
            count_surprise_no_loss_binding += 1
        if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) > 0.35) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) < 0.3):
            count_gain_binding += 1
        if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) > 0.3) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) <= float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key])):
            count_surprise_no_gain_binding += 1

hash_PBM_statistics[str_PBM_name] = (str_PBM_name, len(hash_RBC_SNPs_sliding_window)*8, count_loss_binding, count_gain_binding)
print(hash_PBM_statistics[str_PBM_name])
    
### GATA average for 799 Signalling Centre SNPs

str_PBM_name = "GATA_average"

count_loss_binding = 0
count_surprise_no_loss_binding = 0
count_gain_binding = 0
count_surprise_no_gain_binding = 0

for k, v in hash_combined_SigCtr_SNPs.items():
    for i in range(0, 8):
        allele1_key = v[i][0]
        allele2_key = v[i][1]
        if allele1_key not in hash_all_PBMs_with_Escores[str_PBM_name]:
            print(allele1_key)
            continue
        if allele2_key not in hash_all_PBMs_with_Escores[str_PBM_name]:
            print(allele2_key)
            continue
        if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) > 0.35) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) < 0.3):
            count_loss_binding += 1
        if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) > 0.3) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) <= float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key])):
            count_surprise_no_loss_binding += 1
        if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) > 0.35) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key]) < 0.3):
            count_gain_binding += 1
        if (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) > 0.3) & (float(hash_all_PBMs_with_Escores[str_PBM_name][allele2_key]) <= float(hash_all_PBMs_with_Escores[str_PBM_name][allele1_key])):
            count_surprise_no_gain_binding += 1

hash_PBM_statistics[str_PBM_name] = (str_PBM_name, len(hash_combined_SigCtr_SNPs)*8, count_loss_binding, count_gain_binding)
print(hash_PBM_statistics[str_PBM_name])

('GATA_average', 26104, 276, 181)
('GATA_average', 6392, 91, 67)


In [13]:
print(hash_RBC_SNPs_sliding_window["rs737092"])

tuple_GATA = ("Gata3_1024_contig8mers", "Gata4_8mers_11111111", "Gata5_3768_contig8mers", "Gata6_3769_contig8mers", "GATA_average", "Smad3_3805_contig8mers")
    
for item in tuple_GATA:
    print(item)
#    testing = open(item + "_rs737092" + ".txt", "w")
    for pair in hash_RBC_SNPs_sliding_window["rs737092"]:
        print((hash_all_PBMs_with_Escores[item][pair[0]], hash_all_PBMs_with_Escores[item][pair[1]]))
#        testing.write(pair[0] + "\t" + pair[1] + "\t" + str(hash_all_PBMs_with_Escores[item][pair[0]]) + "\t" + str(hash_all_PBMs_with_Escores[item][pair[1]]) + "\n")
#    testing.close()

[['ATAAGTGC', 'ATAAGTGT'], ['TAAGTGCC', 'TAAGTGTC'], ['AAGTGCCT', 'AAGTGTCT'], ['AGTGCCTG', 'AGTGTCTG'], ['GTGCCTGG', 'GTGTCTGG'], ['TGCCTGGC', 'TGTCTGGC'], ['GCCTGGCT', 'GTCTGGCT'], ['CCTGGCTC', 'TCTGGCTC']]
Gata3_1024_contig8mers
('0.247506381638872', '0.066534966836071')
('-0.130541678029526', '-0.127687491037713')
('-0.140163464295956', '-0.206467939117781')
('-0.089551168682185', '-0.065600802982507')
('-0.025000265536543', '-0.100583090379009')
('-0.192403893641766', '-0.141880860077635')
('-0.324310832842114', '-0.212191649183182')
('-0.283582847447907', '-0.11384335154827')
Gata4_8mers_11111111
('0.26568', '0.10543')
('-0.07743', '0.00538')
('-0.24084', '-0.08602')
('-0.00667', '0.01762')
('0.07895', '0.08521')
('-0.07477', '0.0102')
('0.00253', '-0.02308')
('-0.08513', '0.04092')
Gata5_3768_contig8mers
('0.111846249015859', '0.093296365345699')
('-0.077806208525475', '-0.109344074198021')
('-0.116419947841144', '-0.246190332021395')
('0.013476180172201', '-0.029739046374641')


In [33]:
### Check Framingham gene dataset for T > 0 or < 0 - indicative of cis-eQTL
### Identify number of genes that meet this criteria - 115 such entries, grouped by Entrez ID
### For each of the Entrez IDs, created a list of all the rsIDs that are cis-eQTLs for the respective Entrez IDs
### Created a list of genes (list_of_genes_with_ciseQTL_perturbing_TF_binding) to contain those that were affected by one of the STF rsIDs:
### For every Entrez ID, if one of the 1557 rsIDs_with_altered_binding is present in the list of cis-eQTLs rsIDs of an Entrez ID
### Save that Entrez ID in list_of_genes_with_ciseQTL_perturbing_TF_binding
### We see 86 such unique Entrez IDs in this list.

test_df_Framingham_filtered_gene = pd_df_Framingham.loc[(pd_df_Framingham[22] > 0) | (pd_df_Framingham[22] < 0), : ]
rsIDs_test_df_Framingham_filtered_gene = test_df_Framingham_filtered_gene[0].tolist()
genes_test_df_Framingham_filtered_gene = test_df_Framingham_filtered_gene[18].tolist()

testcount = 0
for i in rsIDs_with_altered_binding:
    if i in rsIDs_test_df_Framingham_filtered_gene:
        testcount += 1
        #print(i)
print(testcount) 
print(len(genes_test_df_Framingham_filtered_gene))
print(len(set(genes_test_df_Framingham_filtered_gene)))

hash_genes_Framingham = {}
for i in range(0, len(genes_test_df_Framingham_filtered_gene)):
    hash_genes_Framingham.setdefault(genes_test_df_Framingham_filtered_gene[i], [])
    (hash_genes_Framingham[genes_test_df_Framingham_filtered_gene[i]]).append(rsIDs_test_df_Framingham_filtered_gene[i])
print(hash_genes_Framingham.keys())

list_of_genes_with_ciseQTL_perturbing_TF_binding = []
for k, v in hash_genes_Framingham.items():
    for i in rsIDs_with_altered_binding:
        if i in v:
            list_of_genes_with_ciseQTL_perturbing_TF_binding.append(k)
print(set(list_of_genes_with_ciseQTL_perturbing_TF_binding))
print(len(set(list_of_genes_with_ciseQTL_perturbing_TF_binding)))
for i in list_of_genes_with_ciseQTL_perturbing_TF_binding:
    if i not in hash_genes_Framingham.keys():
        print(i)

184
457
115
dict_keys(['57412|100528007', 220972, 93550, 51592, 54855, 91181, '3310|3311', 746, 51430, 143686, 81846, 7832, 493, 25778, 388730, 80342, '89894|728543', 9761, 8638, 144406, '6007|6006', 79947, 10162, 28977, 9731, '23499|643314', 4140, 6513, '283635|55012', 1073, 8503, '91612|2877', 10158, 6886, 57143, 374618, '10260|79982|646358', '83440|100287559', '4123|79661|693216', '55466|80349', 55466, 8131, 3681, 146198, 8140, 11216, 92521, 4642, 55876, 6521, '9842|440456', '8742|407977|8741', '9489|8632', 147339, 1777, 4218, 80726, 84902, 51588, '112398|100529264|100652943', 92799, 23645, '81502|100101490', 171023, '100127886|29053', 10018, 6453, 23076, 7327, 64114, 1593, 150275, 8897, 266629, 23761, 25793, 80895, 267020, 3030, 53335, 10678, 5341, 5922, 7037, 8850, 11180, 7318, '2996|2994|2993', 79667, 132660, 345275, 4299, 10826, '6610|285755', 91749, 10767, 64577, 3720, 29116, 9444, '8366|554313|121504|8370|8368|8367|8365|8364|8363|8362|8361|8360|8359|8294', '8364|554313|121504|

In [34]:
### Apply similar workflow from Framingham gene dataset to Framingham exon dataset
### See 107 unique entries out of 144 total entries, but need to account for the "NaN" entries

test_df_Framingham_filtered_exon = exon_pd_df_Framingham.loc[(exon_pd_df_Framingham[22] > 0) | (exon_pd_df_Framingham[22] < 0), : ]
rsIDs_test_df_Framingham_filtered_exon = test_df_Framingham_filtered_exon[0].tolist()
genes_test_df_Framingham_filtered_exon = test_df_Framingham_filtered_exon[18].tolist()

testcount = 0
for i in rsIDs_with_altered_binding:
    if i in rsIDs_test_df_Framingham_filtered_exon:
        testcount += 1
        #print(i)
print(testcount) 
#print((genes_test_df_Framingham_filtered_exon))
print(len(genes_test_df_Framingham_filtered_exon))
print(len(set(genes_test_df_Framingham_filtered_exon)))

hash_exons_Framingham = {}
for i in range(0, len(genes_test_df_Framingham_filtered_exon)):
    hash_exons_Framingham.setdefault(genes_test_df_Framingham_filtered_exon[i], [])
    (hash_exons_Framingham[genes_test_df_Framingham_filtered_exon[i]]).append(rsIDs_test_df_Framingham_filtered_exon[i])
print(hash_exons_Framingham.keys())

list_of_exons_with_ciseQTL_perturbing_TF_binding = []
for k, v in hash_exons_Framingham.items():
    for i in rsIDs_with_altered_binding:
        if i in v:
            list_of_exons_with_ciseQTL_perturbing_TF_binding.append(k)
print(set(list_of_exons_with_ciseQTL_perturbing_TF_binding))
print(len(set(list_of_exons_with_ciseQTL_perturbing_TF_binding)))
for i in list_of_exons_with_ciseQTL_perturbing_TF_binding:
    if i not in hash_exons_Framingham.keys():
        print(i)

235
593
144
dict_keys([22978, 57584, 220972, 93550, 51592, 7812, 54855, nan, 9898, 2213, 746, 92346, 23052, 143686, 81846, 7832, 493, 5929, 25778, 80342, 6311, 9761, 51367, 5715, '6007|6006', 79947, 10162, 11163, '1997|11193', 57470, '23499|643314', 9895, 4140, 6513, 11177, 8503, 4149, '23503|145226', 91748, 6886, 9517, 374618, '10260|79982|646358', '83440|100287559', 10066, '55466|80349', 55466, 8131, '9726|79759', 146198, '497190|283971|348174', 8140, 11216, 92521, 4642, 55876, 7343, '51326|100506084|474170', '100533955|1973', '9489|8632', 147339, 1777, 811, 4218, '54858|23770', 84902, '1938|26812', '112398|100529264|100652943', 23645, 22974, '81502|100101490', 5326, 5335, '5770|140876', '55544|100291105', 10018, 6453, 23076, 7327, 10109, 64114, 1593, 150275, 84164, '284904|730005', 23761, 25793, 4627, 7263, 80895, '91695|253190|27341', 53947, 3030, 53335, '130872|9736', '3921|574040|6044|388524', 10678, 5341, 5922, 23228, 7037, '200958|440993', '116211|255758|100534611', 8850, 51447

In [ ]:
### There were 5 types of nans:
### Manually searched:
# chr 11: 5275530	5275739
# rs11036431		NOT IN rsIDs_with_altered_binding

# chr 11: 5275867	5275903
#rs11036635
#rs7119142
#rs16912979		YES IN
#rs72872555
#rs11036641

# chr 11: 25688127	25688348
#rs669063		NOT IN

#chr 13: 31034028	31034507
#rs9578178
#rs12431309
#rs9508757
#rs9579588
#rs9578179		YES IN

# chr 2: 61343127	61343182
# rs1896293		NOT IN


In [17]:
### Extracting relevant information regarding fine-mapped putative causal variants from Ulirsch Nat Genet 2019

import pandas as pd, os, re

hash_all_fine_mapped_var_0001 = {}
hash_rsIDs_with_PP_1percent = {}
hash_rsIDs_with_PP_0point1 = {}

pd_df_finemapped = pd.read_excel("41588_2019_362_MOESM3_ESM.xlsx", sheetname = 0, header = 2)
list_identifier = pd_df_finemapped["VAR_ID"].tolist()
list_rsID = pd_df_finemapped["rsID"].tolist()
list_PP = pd_df_finemapped["PP"].tolist()

count_PP_001 = 0
count_PP_0001 = 0
list_of_var_PP_001 = []
list_of_var_PP_0001 = []
list_of_rsID_PP_001 = []
list_of_rsID_PP_0001 = []

for i in range(0, len(list_identifier)):
    hash_all_fine_mapped_var_0001[i] = (list_identifier[i], list_rsID[i], list_PP[i])
    if float(list_PP[i]) > 0.01:
        count_PP_001 += 1
        if list_identifier[i] not in list_of_var_PP_001:
            list_of_var_PP_001.append(list_identifier[i])
            if list_rsID[i]:
                list_of_rsID_PP_001.append(list_rsID[i])
    if float(list_PP[i]) > 0.001:
        count_PP_0001 += 1
        if list_identifier[i] not in list_of_var_PP_0001:
            list_of_var_PP_0001.append(list_identifier[i])
            if list_rsID[i]:
                list_of_rsID_PP_0001.append(list_rsID[i])
print(count_PP_001)
print(count_PP_0001)
print(len(list_of_var_PP_001)) ### There are 38654 unique variants that are PP > 0.01
print(len(list_of_var_PP_0001)) ### There are 140739 unique variants that are PP > 0.001
print(len(list_of_rsID_PP_001))
print(len(list_of_rsID_PP_0001))

53843
204599
38654
140739
38654
140739


In [31]:
### Getting list of rsIDs with perturbed TF binding

list_STFs = ['Egr1_2580_contig8mers', 'EGR2_REF_R1_8mers', 'Gli1_v015681_8mers_11111111', 
             'Gli2_v015681_8mers_11111111', 'Gli3_v015681_8mers_11111111', 
             'HES1_R1', 'HES5_R1', 'HES7_R1', 'Rara_1051_contig8mers', 
             'Rarb_8mers_11111111', 'Rarg_8mers_11111111', 'Rxra_1035_contig8mers', 'Rxrg_8mers_11111111', 
             'Smad3_3805_contig8mers', 'TCF4_R1']

list_GATA = ['Gata3_1024_contig8mers', 'Gata4_8mers_11111111', 'Gata5_3768_contig8mers', 'Gata6_3769_contig8mers', 'GATA_average']

rsIDs_with_altered_STF_binding = []
for i in list_rsIDs_PBMs_of_interest:
    if i[0] not in rsIDs_with_altered_STF_binding:
        if i[1] in list_STFs:
            rsIDs_with_altered_STF_binding.append(i[0])
print(len(rsIDs_with_altered_STF_binding))

### There are 1557 rsIDs_with_altered_STF_binding amongst the 3263 SNPs

rsIDs_with_altered_GATA_binding = []
for i in list_rsIDs_PBMs_of_interest:
    if i[0] not in rsIDs_with_altered_GATA_binding:
        if i[1] in list_GATA:
            rsIDs_with_altered_GATA_binding.append(i[0])
print(len(rsIDs_with_altered_GATA_binding))

### There are 448 rsIDs_with_altered_GATA_binding amongst the 3263 SNPs

rsIDs_with_altered_GATA_average_binding = []
for i in list_rsIDs_PBMs_of_interest:
    if i[0] not in rsIDs_with_altered_GATA_average_binding:
        if i[1] == "GATA_average":
            rsIDs_with_altered_GATA_average_binding.append(i[0])
print(len(rsIDs_with_altered_GATA_average_binding))

1557
448
184


In [32]:
### Considering intersection between 3263 SNPs and fine-mapped putative causal variants from Ulirsch Nat Genet 2019

count_rsIDs_PP_001_altered_STF_binding = 0
count_rsIDs_PP_0001_altered_STF_binding = 0
count_rsIDs_PP_001_altered_GATA_binding = 0
count_rsIDs_PP_0001_altered_GATA_binding = 0
count_rsIDs_PP_001_altered_GATA_average_binding = 0
count_rsIDs_PP_0001_altered_GATA_average_binding = 0
count_rsIDs_PP_001_all = 0
count_rsIDs_PP_0001_all = 0

#file_to_write_rsIDs_PP_001_all = open("rsIDs_PP_001_all.txt", "w")
#file_to_write_rsIDs_PP_0001_all = open("rsIDs_PP_0001_all.txt", "w")

for i in rsIDs_with_altered_STF_binding:
    if i in list_of_rsID_PP_001:
        count_rsIDs_PP_001_altered_STF_binding += 1
    if i in list_of_rsID_PP_0001:
        count_rsIDs_PP_0001_altered_STF_binding += 1

for i in rsIDs_with_altered_GATA_binding:
    if i in list_of_rsID_PP_001:
        count_rsIDs_PP_001_altered_GATA_binding += 1
    if i in list_of_rsID_PP_0001:
        count_rsIDs_PP_0001_altered_GATA_binding += 1

for i in rsIDs_with_altered_GATA_average_binding:
    if i in list_of_rsID_PP_001:
        count_rsIDs_PP_001_altered_GATA_average_binding += 1
    if i in list_of_rsID_PP_0001:
        count_rsIDs_PP_0001_altered_GATA_average_binding += 1

for j in hash_RBC_SNPs_sliding_window.keys():
    if j in list_of_rsID_PP_001:
        count_rsIDs_PP_001_all += 1
#        file_to_write_rsIDs_PP_001_all.write(j + "\n")
    if j in list_of_rsID_PP_0001:
        count_rsIDs_PP_0001_all += 1
#        file_to_write_rsIDs_PP_0001_all.write(j + "\n")

#file_to_write_rsIDs_PP_001_all.close()
#file_to_write_rsIDs_PP_0001_all.close()
print(count_rsIDs_PP_001_altered_STF_binding) # 542 for STFs, PP > 0.01
print(count_rsIDs_PP_0001_altered_STF_binding) # 911 for STFs, PP > 0.001
print(count_rsIDs_PP_001_altered_GATA_binding) # 158 for GATA, PP > 0.01
print(count_rsIDs_PP_0001_altered_GATA_binding) # 276 for GATA, PP > 0.001
print(count_rsIDs_PP_001_altered_GATA_average_binding) # 62 for GATA_average, PP > 0.01
print(count_rsIDs_PP_0001_altered_GATA_average_binding) # 115 for GATA_average, PP > 0.001
print(count_rsIDs_PP_001_all) # 1064 for all 3263 rsIDs
print(count_rsIDs_PP_0001_all) # 1911 for all 3263 rsIDs

### These values can be used to perform Fisher's exact tests (two-sided) to look at intersection of
### the 3263 single nucleotide substitution SNPs in this study and fine-mapped blood trait SNPs.

542
911
158
276
62
115
1064
1911
